# 1. Mô tả bộ dữ liệu
### Nguồn dữ liệu:
Bộ dữ liệu Recent Earthquakes được thu thập từ USGS (United States Geological Survey), cung cấp thông tin chi tiết về các trận động đất gần đây trên toàn thế giới. Dữ liệu bao gồm vị trí địa lý, độ sâu, cường độ, loại cảnh báo, và nhiều đặc trưng địa lý – hành chính kèm theo.
### Mục tiêu bài toán:
Phân tích và dự đoán cường độ động đất (magnitude) dựa trên các đặc trưng liên quan đến vị trí địa lý, độ sâu, và các thông số kỹ thuật khác.

→ Đây là bài toán hồi quy (Regression), trong đó biến mục tiêu (target) là magnitude.

### Quy mô dữ liệu:
- Số lượng mẫu: 1,137 bản ghi.

- Số lượng thuộc tính: 40+ cột, bao gồm thông tin kỹ thuật, vị trí, và siêu dữ liệu.

- Không có giá trị thiếu đáng kể ở các thuộc tính chính (như magnitude, latitude, longitude, depth).

- Một số cột có tỷ lệ thiếu cao (ví dụ: postcode có hơn 80% giá trị NaN).
### Các thuộc tính chính:
| Thuộc tính           | Mô tả                                              | Kiểu dữ liệu             |
| -------------------- | -------------------------------------------------- | ------------------------ |
| **id**               | Mã định danh duy nhất của mỗi trận động đất        | Object                   |
| **magnitude**        | Cường độ của trận động đất theo thang Richter      | Float (Target)           |
| **type**             | Loại sự kiện (hầu hết là “earthquake”)             | Categorical              |
| **latitude**         | Vĩ độ tâm chấn                                     | Float                    |
| **longitude**        | Kinh độ tâm chấn                                   | Float                    |
| **depth**            | Độ sâu của tâm chấn (km)                           | Float                    |
| **magType**          | Loại phép đo cường độ (mww, ml, mb, …)             | Categorical              |
| **time**             | Thời gian xảy ra (Unix timestamp)                  | Integer                  |
| **alert**            | Mức cảnh báo (green, yellow, orange, red)          | Categorical (có missing) |
| **continent**        | Châu lục xảy ra động đất                           | Categorical              |
| **country**          | Quốc gia xảy ra động đất                           | Categorical              |
| **location / place** | Mô tả vị trí địa lý chi tiết                       | Text                     |
| **tsunami**          | Cờ đánh dấu có xuất hiện sóng thần hay không (0/1) | Integer (Binary)         |
| **sig**              | Mức độ nghiêm trọng (severity index)               | Integer                  |
| **gap / rms / dmin** | Các tham số đo độ chính xác của vị trí địa chấn    | Float                    |



### Đặc điểm dữ liệu:

- Dữ liệu đa dạng về không gian (các trận động đất diễn ra ở nhiều khu vực trên thế giới: Mỹ, Nhật Bản, Đài Loan, Indonesia, Chile, v.v.).

- Cột magnitude dao động trong khoảng 3.0 – 7.6, phản ánh cả động đất nhỏ và trung bình – lớn.

- Các cột như alert, continent, country có thể được mã hóa one-hot hoặc Label Encoding cho mô hình.

- Một số cột chứa chuỗi hoặc cấu trúc JSON (locationDetails, sources, types) – nên được loại bỏ hoặc trích rút đặc trưng.

- Các giá trị định lượng có quy mô rất khác nhau (ví dụ depth so với rms), cần chuẩn hóa (scaling) trước khi huấn luyện.

### Chất lượng và xử lý dữ liệu:

- Giá trị thiếu:

  - Cột postcode thiếu nhiều (>80%), có thể loại bỏ.

  - continent, country, alert có giá trị NaN, có thể điền bằng “unknown”.

- Chuẩn hóa dữ liệu số:

  - Áp dụng StandardScaler hoặc MinMaxScaler cho các thuộc tính như depth, latitude, longitude, sig, rms.

- Mã hóa dữ liệu phân loại:

  - alert, magType, continent, country → LabelEncoder hoặc OneHotEncoder.

- Loại bỏ cột trùng lặp hoặc siêu dữ liệu không cần thiết:

  - Ví dụ: id, url, detailUrl, what3words, locationDetails, sources.

### Ứng dụng và phạm vi sử dụng:

- Phân tích dữ liệu địa chấn: Nhận diện khu vực có hoạt động địa chấn mạnh.

- Dự báo nguy cơ động đất: Xây dựng mô hình hồi quy dự đoán magnitude hoặc xác suất sóng thần (tsunami).

- Nghiên cứu khoa học: Dùng làm bộ dữ liệu thực tế trong học máy về hồi quy không gian – thời gian.

- Ứng dụng mô hình:

  - Linear Regression

  - Random Forest Regressor

  - XGBoost / Gradient Boosting

  - Support Vector Regressor (SVR)

  - Neural Network Regressor

# 2. Quy trình chia dữ liệu và huấn luyện mô hình Linear Regression

## 2.1 Tải và đọc dữ liệu

In [ ]:
import kagglehub
import pandas as pd
import glob
import os

# Tải dữ liệu từ kagglehub
try:
    path = kagglehub.dataset_download("shreyasur965/recent-earthquakes")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}")
    exit()

csv_files = glob.glob(os.path.join(path, '*.csv'))

if not csv_files:
    print("Không tìm thấy file CSV nào trong thư mục đã tải.")
    exit()

data_file_path = csv_files[0]
df = pd.read_csv(data_file_path)

print("  PHÂN TÍCH CÁC NHÃN/GIÁ TRỊ ĐỘC NHẤT TRONG DỮ LIỆU  ")

for col in df.columns:
    print(f"\n--- Cột: {col} (Dtype: {df[col].dtype}) ---")

    if df[col].dtype == 'object':
        print("Các nhãn/giá trị độc nhất và tần suất:")
        print(df[col].value_counts(dropna=False).head(10))
        print(f"Tổng số giá trị độc nhất: {df[col].nunique()}")

    elif df[col].dtype == 'int64':
        if df[col].nunique() <= 10:
            print("Các giá trị/nhãn số nguyên và tần suất:")
            print(df[col].value_counts(dropna=False))
        else:
            print(f"Số lượng giá trị độc nhất: {df[col].nunique()}. "
                  f"Giá trị nhỏ nhất: {df[col].min()}, Giá trị lớn nhất: {df[col].max()}.")

    elif df[col].dtype == 'float64':
        print(f"Số lượng giá trị độc nhất: {df[col].nunique()}. "
              f"Giá trị nhỏ nhất: {df[col].min()}, Giá trị lớn nhất: {df[col].max()}.")
        print(f"Tỷ lệ giá trị thiếu (NaN): {df[col].isnull().sum() / len(df) * 100:.2f}%")

print("THÔNG TIN TỔNG QUAN DỮ LIỆU:")
print(df.info())
print("\n5 dòng đầu tiên:")
print(df.head())

Using Colab cache for faster access to the 'recent-earthquakes' dataset.
  PHÂN TÍCH CÁC NHÃN/GIÁ TRỊ ĐỘC NHẤT TRONG DỮ LIỆU  

--- Cột: id (Dtype: object) ---
Các nhãn/giá trị độc nhất và tần suất:
id
us6000mirj    4
us6000nnuf    4
us7000ndte    4
tx2024rcoz    4
tx2024qyag    4
tx2024qnpx    4
us7000n8wd    4
tx2024scvz    4
us6000kyb4    4
us6000kr3t    4
Name: count, dtype: int64
Tổng số giá trị độc nhất: 602

--- Cột: magnitude (Dtype: float64) ---
Số lượng giá trị độc nhất: 86. Giá trị nhỏ nhất: 3.0, Giá trị lớn nhất: 7.6.
Tỷ lệ giá trị thiếu (NaN): 0.00%

--- Cột: type (Dtype: object) ---
Các nhãn/giá trị độc nhất và tần suất:
type
earthquake    1137
Name: count, dtype: int64
Tổng số giá trị độc nhất: 1

--- Cột: title (Dtype: object) ---
Các nhãn/giá trị độc nhất và tần suất:
title
M 5.5 - Izu Islands, Japan region             12
M 5.7 - Izu Islands, Japan region             12
M 5.3 - Izu Islands, Japan region             12
M 5.4 - Izu Islands, Japan region             12
M 

## 2.2 Kiểm tra thông tin và xử lý dữ liệu

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Giả định dữ liệu đã tải sẵn vào biến df
print("THÔNG TIN TỔNG QUAN:")
df.info()

print("\nKIỂM TRA GIÁ TRỊ THIẾU:")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# Giữ lại một số cột phù hợp cho bài toán hồi quy (magnitude là biến mục tiêu)
columns_to_keep = [
    'depth', 'latitude', 'longitude', 'rms', 'gap', 'dmin',
    'sig', 'nst', 'distanceKM', 'tsunami', 'mmi', 'cdi', 'magnitude'
]
df = df[columns_to_keep].copy()

# Xử lý giá trị thiếu
df = df.fillna(df.median(numeric_only=True))

# Loại bỏ hàng bị thiếu nghiêm trọng
df = df.dropna()

# Kiểm tra kiểu dữ liệu
print("\nDỮ LIỆU SAU KHI XỬ LÝ:")
print(df.describe().T)

# Biến mục tiêu
target_col = 'magnitude'


THÔNG TIN TỔNG QUAN:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 43 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1137 non-null   object 
 1   magnitude        1137 non-null   float64
 2   type             1137 non-null   object 
 3   title            1137 non-null   object 
 4   date             1137 non-null   object 
 5   time             1137 non-null   int64  
 6   updated          1137 non-null   int64  
 7   url              1137 non-null   object 
 8   detailUrl        1137 non-null   object 
 9   felt             1137 non-null   int64  
 10  cdi              1137 non-null   int64  
 11  mmi              1137 non-null   int64  
 12  alert            764 non-null    object 
 13  status           1137 non-null   object 
 14  tsunami          1137 non-null   int64  
 15  sig              1137 non-null   int64  
 16  net              1137 non-null   object

## 2.3 Chia dữ liệu

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Tách biến đầu vào và mục tiêu
X = df.drop(columns=[target_col])
y = df[target_col]

# Chia 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Kích thước tập huấn luyện:", X_train.shape)
print("Kích thước tập kiểm tra:", X_test.shape)


Kích thước tập huấn luyện: (795, 12)
Kích thước tập kiểm tra: (342, 12)


## 2.4 Huấn luyện mô hình Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Khởi tạo và huấn luyện
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Dự đoán
y_pred = lr_model.predict(X_test)

## 2.5 Đánh giá mô hình

In [ ]:
# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("   KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH LINEAR REGRESSION")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.3f}")

# In hệ số hồi quy
print("\nHệ số hồi quy (Coefficients):")
for name, coef in zip(df.drop(columns=['magnitude']).columns, lr_model.coef_):
    print(f"{name}: {coef:.3f}")
print(f"\nHệ số chặn (Intercept): {lr_model.intercept_:.3f}")

   KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH LINEAR REGRESSION
Mean Squared Error (MSE): 0.11
R² Score: 0.894

Hệ số hồi quy (Coefficients):
depth: 0.117
latitude: -0.139
longitude: 0.383
rms: 0.208
gap: 0.034
dmin: 0.104
sig: 0.271
nst: 0.106
distanceKM: 0.098
tsunami: 0.141
mmi: 0.130
cdi: 0.062

Hệ số chặn (Intercept): 4.884
